In [11]:
# Cannabis PhenoHunter Pro - Enhanced Edition (Fixed & Complete)
# Author: @Hosuay
# Version: 2.3
# Last Updated: 2025-10-20 (Full COA Import + Database Management)

!pip install -q requests beautifulsoup4 plotly torch scikit-learn selenium pandas numpy fuzzywuzzy python-levenshtein pdfplumber tabula-py

import os, glob, pandas as pd, numpy as np, torch, torch.nn as nn, torch.optim as optim
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pickle, json, time, re
from fuzzywuzzy import process
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
from sklearn.base import BaseEstimator
from datetime import datetime
import requests
from bs4 import BeautifulSoup

print("=" * 70)
print("CANNABIS PHENO HUNTER PRO - ENHANCED EDITION")
print("Advanced Trait-Based Breeding & Real Data Integration")
print("=" * 70)

# ======================================================================
# SAMPLE STRAIN DATABASE
# ======================================================================

sample_strains = {
    'Blue Dream': {'type': 'hybrid', 'thc_pct': 19.5, 'cbd_pct': 0.8, 'cbg_pct': 0.9, 'myrcene_pct': 1.2, 'pinene_pct': 0.8, 'limonene_pct': 1.5, 'caryophyllene_pct': 0.9, 'linalool_pct': 0.4, 'humulene_pct': 0.3, 'cbc_pct': 0.2, 'cbda_pct': 0.15},
    'OG Kush': {'type': 'hybrid', 'thc_pct': 22.5, 'cbd_pct': 0.3, 'cbg_pct': 0.6, 'myrcene_pct': 1.8, 'limonene_pct': 1.2, 'caryophyllene_pct': 1.4, 'pinene_pct': 0.7, 'linalool_pct': 0.6, 'humulene_pct': 0.5, 'cbc_pct': 0.25, 'cbda_pct': 0.1},
    'Sour Diesel': {'type': 'sativa', 'thc_pct': 20.8, 'cbd_pct': 0.2, 'cbg_pct': 0.7, 'limonene_pct': 2.1, 'myrcene_pct': 0.8, 'pinene_pct': 1.3, 'caryophyllene_pct': 0.9, 'linalool_pct': 0.3, 'humulene_pct': 0.4, 'cbc_pct': 0.18, 'cbda_pct': 0.12},
    'Girl Scout Cookies': {'type': 'hybrid', 'thc_pct': 24.2, 'cbd_pct': 0.5, 'cbg_pct': 0.8, 'caryophyllene_pct': 1.6, 'limonene_pct': 1.4, 'myrcene_pct': 1.1, 'linalool_pct': 0.7, 'pinene_pct': 0.6, 'humulene_pct': 0.45, 'cbc_pct': 0.3, 'cbda_pct': 0.2},
    'Granddaddy Purple': {'type': 'indica', 'thc_pct': 21.5, 'cbd_pct': 0.9, 'cbg_pct': 0.5, 'myrcene_pct': 2.4, 'caryophyllene_pct': 1.5, 'linalool_pct': 1.0, 'pinene_pct': 0.5, 'limonene_pct': 0.7, 'humulene_pct': 0.6, 'cbc_pct': 0.22, 'cbda_pct': 0.18},
    'Wedding Cake': {'type': 'indica', 'thc_pct': 25.3, 'cbd_pct': 0.4, 'cbg_pct': 0.9, 'limonene_pct': 1.8, 'caryophyllene_pct': 1.7, 'myrcene_pct': 1.3, 'linalool_pct': 0.8, 'pinene_pct': 0.7, 'humulene_pct': 0.55, 'cbc_pct': 0.28, 'cbda_pct': 0.16},
    'Gelato': {'type': 'hybrid', 'thc_pct': 23.8, 'cbd_pct': 0.6, 'cbg_pct': 0.85, 'caryophyllene_pct': 1.5, 'myrcene_pct': 1.4, 'limonene_pct': 1.6, 'linalool_pct': 0.65, 'pinene_pct': 0.8, 'humulene_pct': 0.5, 'cbc_pct': 0.26, 'cbda_pct': 0.19},
    'Northern Lights': {'type': 'indica', 'thc_pct': 18.9, 'cbd_pct': 1.2, 'cbg_pct': 0.6, 'myrcene_pct': 2.1, 'caryophyllene_pct': 1.3, 'pinene_pct': 0.9, 'linalool_pct': 0.9, 'limonene_pct': 0.8, 'humulene_pct': 0.55, 'cbc_pct': 0.2, 'cbda_pct': 0.14},
    'Jack Herer': {'type': 'sativa', 'thc_pct': 20.3, 'cbd_pct': 0.4, 'cbg_pct': 0.75, 'pinene_pct': 1.6, 'limonene_pct': 1.9, 'myrcene_pct': 0.9, 'caryophyllene_pct': 1.0, 'linalool_pct': 0.4, 'humulene_pct': 0.45, 'cbc_pct': 0.21, 'cbda_pct': 0.13},
    'Durban Poison': {'type': 'sativa', 'thc_pct': 18.5, 'cbd_pct': 0.2, 'cbg_pct': 0.9, 'limonene_pct': 2.0, 'pinene_pct': 1.5, 'myrcene_pct': 0.7, 'caryophyllene_pct': 0.85, 'linalool_pct': 0.3, 'humulene_pct': 0.35, 'cbc_pct': 0.17, 'cbda_pct': 0.1}
}

# Convert to DataFrame
df_strains = []
for name, profile in sample_strains.items():
    row = {'strain_name': name, 'type': profile['type']}
    row.update({f'{k}': v for k, v in profile.items() if k != 'type'})
    df_strains.append(row)

df = pd.DataFrame(df_strains)

# Define feature columns
trained_feature_columns = ['thc_pct', 'cbd_pct', 'cbda_pct', 'cbg_pct', 'cbc_pct', 'myrcene_pct', 'limonene_pct', 'pinene_pct', 'linalool_pct', 'caryophyllene_pct', 'humulene_pct']

# Ensure all columns exist
for col in trained_feature_columns:
    if col not in df.columns:
        df[col] = 0.0

X = df[trained_feature_columns].fillna(0)
X_tensor = torch.tensor(X.values, dtype=torch.float32)

# ======================================================================
# AUTOENCODER FOR OFFSPRING GENERATION
# ======================================================================

class Autoencoder(nn.Module):
    def __init__(self, input_dim=11, latent_dim=5):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 8),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(8, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 8),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(8, input_dim),
            nn.Softplus()
        )

    def forward(self, x):
        latent = self.encoder(x)
        return self.decoder(latent)

ae = Autoencoder(input_dim=len(trained_feature_columns))
criterion = nn.MSELoss()
optimizer = optim.Adam(ae.parameters(), lr=0.005, weight_decay=1e-5)

print(f"✓ Loaded {len(df)} premium strains with complete chemical profiles")
print(f"✓ Average THC: {X['thc_pct'].mean():.2f}%, CBD: {X['cbd_pct'].mean():.2f}%")
print("\nTraining autoencoder...")

for epoch in range(200):
    optimizer.zero_grad()
    output = ae(X_tensor)
    loss = criterion(output, X_tensor)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 50 == 0:
        print(f"  Epoch {epoch+1}/200 - Loss: {loss.item():.4f}")

print("✓ Autoencoder trained successfully")

# ======================================================================
# TRAIT OPTIMIZER
# ======================================================================

class TraitOptimizer:
    def __init__(self, strain_db):
        self.strain_db = strain_db
        self.trait_compounds = {
            'potency': {'compounds': ['thc_pct', 'cbg_pct'], 'weights': [0.8, 0.2]},
            'pain_relief': {'compounds': ['cbd_pct', 'myrcene_pct', 'caryophyllene_pct'], 'weights': [0.5, 0.3, 0.2]},
            'anxiety_relief': {'compounds': ['cbd_pct', 'linalool_pct', 'myrcene_pct'], 'weights': [0.4, 0.4, 0.2]},
            'focus': {'compounds': ['thc_pct', 'pinene_pct', 'limonene_pct'], 'weights': [0.4, 0.3, 0.3]},
            'sleep': {'compounds': ['cbc_pct', 'myrcene_pct', 'linalool_pct'], 'weights': [0.3, 0.4, 0.3]},
            'inflammation': {'compounds': ['cbd_pct', 'caryophyllene_pct', 'humulene_pct'], 'weights': [0.4, 0.4, 0.2]}
        }

    def calculate_trait_score(self, strain_profile, trait):
        if trait not in self.trait_compounds:
            return 0
        trait_info = self.trait_compounds[trait]
        compounds = trait_info['compounds']
        weights = trait_info['weights']
        score = sum(strain_profile[comp] * weight for comp, weight in zip(compounds, weights))
        return score

    def find_optimal_partners(self, main_strain, desired_trait, top_n=5):
        main_profile = self.strain_db[self.strain_db['strain_name'] == main_strain]
        if main_profile.empty:
            return []

        current_trait_score = self.calculate_trait_score(main_profile.iloc[0], desired_trait)
        scores = []

        for _, strain in self.strain_db.iterrows():
            if strain['strain_name'] == main_strain:
                continue

            partner_trait_score = self.calculate_trait_score(strain, desired_trait)
            improvement_potential = max(0, partner_trait_score - current_trait_score)
            diversity = 1 - cosine_similarity(
                main_profile[trained_feature_columns].values.reshape(1, -1),
                strain[trained_feature_columns].values.reshape(1, -1)
            )[0][0]
            stability = 1 / (1 + np.std([strain[c] for c in trained_feature_columns]))
            final_score = (improvement_potential * 0.5 + diversity * 0.3 + stability * 0.2)
            scores.append((strain['strain_name'], final_score))

        return sorted(scores, key=lambda x: x[1], reverse=True)[:top_n]

    def predict_offspring_traits(self, parent1_profile, parent2_profile, parent1_weight=0.6):
        traits = {}
        parent2_weight = 1 - parent1_weight

        for trait, info in self.trait_compounds.items():
            parent1_score = self.calculate_trait_score(parent1_profile, trait)
            parent2_score = self.calculate_trait_score(parent2_profile, trait)
            variation = np.random.uniform(0.9, 1.1)
            predicted_score = (parent1_score * parent1_weight + parent2_score * parent2_weight) * variation
            traits[trait] = predicted_score

        return traits

# ======================================================================
# BREEDING STRATEGY
# ======================================================================

class AdvancedBreedingStrategy:
    def __init__(self, strain_db, autoencoder):
        self.strain_db = strain_db
        self.ae = autoencoder
        self.trait_optimizer = TraitOptimizer(strain_db)

    def generate_offspring(self, main_parent, partner, trait_bias=0.7):
        main_idx = self.strain_db[self.strain_db['strain_name'] == main_parent].index[0]
        partner_idx = self.strain_db[self.strain_db['strain_name'] == partner].index[0]

        parent_vecs = X_tensor[[main_idx, partner_idx]]
        latent_vecs = self.ae.encoder(parent_vecs)

        weights = torch.tensor([trait_bias, 1-trait_bias], dtype=torch.float32).unsqueeze(1)
        combined_latent = (latent_vecs * weights).sum(dim=0) / weights.sum()

        noise = torch.randn_like(combined_latent) * 0.05
        combined_latent += noise

        return self.ae.decoder(combined_latent.unsqueeze(0))

    def optimize_cross(self, main_parent, partner, desired_trait):
        offspring_base = self.generate_offspring(main_parent, partner)

        main_profile = self.strain_db[self.strain_db['strain_name'] == main_parent].iloc[0]
        partner_profile = self.strain_db[self.strain_db['strain_name'] == partner].iloc[0]

        predicted_traits = self.trait_optimizer.predict_offspring_traits(main_profile, partner_profile)

        return {
            'chemical_profile': offspring_base.detach().numpy(),
            'predicted_traits': predicted_traits
        }

# ======================================================================
# VISUALIZER
# ======================================================================

class StrainVisualizer:
    @staticmethod
    def plot_trait_comparison(main_parent, partners_data, trait_optimizer):
        fig = go.Figure()
        traits = list(trait_optimizer.trait_compounds.keys())

        main_profile = df[df['strain_name'] == main_parent].iloc[0]
        main_scores = [trait_optimizer.calculate_trait_score(main_profile, t) for t in traits]

        fig.add_trace(go.Scatterpolar(
            r=main_scores,
            theta=traits,
            fill='toself',
            name=f'{main_parent} (Main)',
            line=dict(color='#667eea', width=3)
        ))

        colors = ['#43e97b', '#fa709a', '#f093fb']
        for i, (partner, score) in enumerate(partners_data):
            partner_profile = df[df['strain_name'] == partner].iloc[0]
            partner_scores = [trait_optimizer.calculate_trait_score(partner_profile, t) for t in traits]
            fig.add_trace(go.Scatterpolar(
                r=partner_scores,
                theta=traits,
                fill='toself',
                name=f'{partner} (Score: {score:.2f})',
                line=dict(color=colors[i], width=2)
            ))

        fig.update_layout(
            polar=dict(radialaxis=dict(visible=True, range=[0, 30])),
            showlegend=True,
            title={'text': "Trait Comparison: Main Parent vs Potential Partners", 'x': 0.5, 'xanchor': 'center', 'font': {'size': 20}},
            height=600
        )
        return fig

    @staticmethod
    def plot_chemical_profile(strain_data, title="Chemical Profile"):
        categories = [
            ('Cannabinoids', ['thc_pct', 'cbd_pct', 'cbda_pct', 'cbg_pct', 'cbc_pct']),
            ('Terpenes', ['myrcene_pct', 'limonene_pct', 'pinene_pct', 'linalool_pct', 'caryophyllene_pct', 'humulene_pct'])
        ]

        fig = go.Figure()

        for category, compounds in categories:
            values = [strain_data[c] for c in compounds]
            labels = [c.replace('_pct', '').upper() for c in compounds]
            fig.add_trace(go.Bar(
                name=category,
                x=labels,
                y=values,
                text=[f"{v:.2f}%" for v in values],
                textposition='auto'
            ))

        fig.update_layout(
            title={'text': title, 'x': 0.5, 'xanchor': 'center', 'font': {'size': 20}},
            barmode='group',
            height=500,
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        return fig

# ======================================================================
# COA PARSER & DATABASE MANAGER
# ======================================================================

class COAParser:
    @staticmethod
    def parse_csv(file_path):
        try:
            df_new = pd.read_csv(file_path)
            print(f"✓ Loaded {len(df_new)} strains from CSV")
            return df_new
        except Exception as e:
            print(f"❌ Error reading CSV: {e}")
            return None

    @staticmethod
    def parse_json(file_path):
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)

            if isinstance(data, list):
                df_new = pd.DataFrame(data)
            else:
                df_new = pd.DataFrame([data])

            print(f"✓ Loaded {len(df_new)} strains from JSON")
            return df_new

        except Exception as e:
            print(f"❌ Error reading JSON: {e}")
            return None

    @staticmethod
    def parse_pdf_coa(file_path):
        try:
            import pdfplumber

            data = {
                'strain_name': None,
                'type': 'unknown',
                'thc_pct': 0.0,
                'cbd_pct': 0.0,
                'cbg_pct': 0.0,
                'cbc_pct': 0.0,
                'cbda_pct': 0.0
            }

            terpenes = ['myrcene', 'limonene', 'pinene', 'linalool', 'caryophyllene', 'humulene']
            for terp in terpenes:
                data[f'{terp}_pct'] = 0.0

            with pdfplumber.open(file_path) as pdf:
                text = ""
                for page in pdf.pages:
                    text += page.extract_text() + "\n"

                cannabinoid_patterns = {
                    'thc': r'THC[:\s]*(\d+\.?\d*)%?',
                    'cbd': r'CBD[:\s]*(\d+\.?\d*)%?',
                    'cbg': r'CBG[:\s]*(\d+\.?\d*)%?',
                    'cbc': r'CBC[:\s]*(\d+\.?\d*)%?',
                    'cbda': r'CBDA[:\s]*(\d+\.?\d*)%?'
                }

                for compound, pattern in cannabinoid_patterns.items():
                    match = re.search(pattern, text, re.IGNORECASE)
                    if match:
                        data[f'{compound}_pct'] = float(match.group(1))

                for terp in terpenes:
                    pattern = rf'{terp}[:\s]*(\d+\.?\d*)%?'
                    match = re.search(pattern, text, re.IGNORECASE)
                    if match:
                        data[f'{terp}_pct'] = float(match.group(1))

                name_patterns = [
                    r'Strain[:\s]*([A-Za-z\s]+)',
                    r'Product[:\s]*([A-Za-z\s]+)',
                    r'Sample[:\s]*([A-Za-z\s]+)'
                ]
                for pattern in name_patterns:
                    match = re.search(pattern, text, re.IGNORECASE)
                    if match:
                        data['strain_name'] = match.group(1).strip()
                        break

            print(f"✓ Parsed PDF COA: {data['strain_name']}")
            return pd.DataFrame([data])

        except Exception as e:
            print(f"❌ Error parsing PDF: {e}")
            return None

class StrainDatabase:
    def __init__(self, initial_df):
        self.df = initial_df.copy()
        self.original_size = len(initial_df)
        self.custom_strains = []

    def add_strain(self, strain_data):
        for col in trained_feature_columns:
            if col not in strain_data:
                strain_data[col] = 0.0

        if 'type' not in strain_data:
            strain_data['type'] = 'unknown'

        new_row = pd.DataFrame([strain_data])
        self.df = pd.concat([self.df, new_row], ignore_index=True)
        self.custom_strains.append(strain_data['strain_name'])

        print(f"✓ Added strain: {strain_data['strain_name']}")
        return True

    def import_from_file(self, file_path, file_type='auto'):
        parser = COAParser()

        if file_type == 'auto':
            if file_path.endswith('.csv'):
                file_type = 'csv'
            elif file_path.endswith('.json'):
                file_type = 'json'
            elif file_path.endswith('.pdf'):
                file_type = 'pdf'
            else:
                print("❌ Unsupported file type")
                return False

        if file_type == 'csv':
            new_data = parser.parse_csv(file_path)
        elif file_type == 'json':
            new_data = parser.parse_json(file_path)
        elif file_type == 'pdf':
            new_data = parser.parse_pdf_coa(file_path)
        else:
            print("❌ Invalid file type")
            return False

        if new_data is None or new_data.empty:
            return False

        for col in trained_feature_columns:
            if col not in new_data.columns:
                new_data[col] = 0.0

        if 'type' not in new_data.columns:
            new_data['type'] = 'unknown'

        initial_count = len(self.df)
        self.df = pd.concat([self.df, new_data], ignore_index=True)
        self.df = self.df.drop_duplicates(subset=['strain_name'], keep='last')

        added_count = len(self.df) - initial_count
        self.custom_strains.extend(new_data['strain_name'].tolist())

        print(f"✓ Added {added_count} new strains")
        print(f"📊 Total strains: {len(self.df)}")

        return True

    def export_database(self, file_path='my_strain_database.csv'):
        self.df.to_csv(file_path, index=False)
        print(f"✓ Database exported to {file_path}")
        return file_path

    def validate_cross_accuracy(self, parent1, parent2, offspring_name):
        if offspring_name not in self.df['strain_name'].values:
            print(f"❌ Offspring not found in database")
            return None

        actual_offspring = self.df[self.df['strain_name'] == offspring_name].iloc[0]

        breeder = AdvancedBreedingStrategy(self.df, ae)
        result = breeder.optimize_cross(parent1, parent2, 'potency')
        predicted_profile = pd.DataFrame(result['chemical_profile'], columns=trained_feature_columns)

        accuracy_scores = {}
        print(f"\n{'='*70}")
        print(f"CROSS VALIDATION: {parent1} × {parent2} → {offspring_name}")
        print(f"{'='*70}")
        print(f"\n{'Compound':<15} {'Actual':<10} {'Predicted':<10} {'Error':<10} {'Accuracy'}")
        print("-"*70)

        for col in trained_feature_columns:
            actual_val = actual_offspring[col]
            predicted_val = predicted_profile.iloc[0][col]
            error = abs(actual_val - predicted_val)

            if actual_val > 0:
                accuracy = max(0, 100 - (error / actual_val * 100))
            else:
                accuracy = 100 if predicted_val < 0.5 else 0

            accuracy_scores[col] = accuracy

            compound = col.replace('_pct', '').upper()
            print(f"{compound:<15} {actual_val:<10.2f} {predicted_val:<10.2f} {error:<10.2f} {accuracy:.1f}%")

        overall_accuracy = np.mean(list(accuracy_scores.values()))
        print(f"\n{'='*70}")
        print(f"OVERALL PREDICTION ACCURACY: {overall_accuracy:.2f}%")
        print(f"{'='*70}")

        return {
            'offspring_name': offspring_name,
            'overall_accuracy': overall_accuracy,
            'compound_accuracies': accuracy_scores
        }

strain_db = StrainDatabase(df)
print("✓ Strain database manager initialized")

# ======================================================================
# UI COMPONENTS WITH PROFESSIONAL STYLING
# ======================================================================

# Professional CSS styling
custom_css = """
<style>
    .pheno-section {
        background: white;
        padding: 20px;
        border-radius: 12px;
        margin: 15px 0;
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        border-left: 4px solid #667eea;
    }
    .pheno-header {
        font-size: 18px;
        font-weight: 600;
        color: #2d3748;
        margin-bottom: 15px;
        display: flex;
        align-items: center;
    }
    .pheno-icon {
        font-size: 24px;
        margin-right: 10px;
    }
    .pheno-description {
        color: #718096;
        font-size: 14px;
        margin-bottom: 15px;
        line-height: 1.5;
    }
    .widget-label > label {
        font-weight: 500 !important;
        color: #4a5568 !important;
    }
</style>
"""

title_html = widgets.HTML(f"""
{custom_css}
<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 30px; border-radius: 15px; margin-bottom: 25px;
            box-shadow: 0 8px 16px rgba(102, 126, 234, 0.4);'>
    <div style='display: flex; align-items: center; justify-content: space-between;'>
        <div>
            <h1 style='color: white; margin: 0; font-size: 36px; font-weight: 700;'>
                🧬 Cannabis PhenoHunter Pro
            </h1>
            <p style='color: #e0e7ff; margin: 10px 0 0 0; font-size: 16px; font-weight: 400;'>
                Advanced AI-Powered Cannabis Breeding Platform
            </p>
        </div>
        <div style='background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px;'>
            <div style='color: white; font-size: 14px; font-weight: 600;'>Database</div>
            <div style='color: #e0e7ff; font-size: 28px; font-weight: 700;'>{len(strain_db.df)}</div>
            <div style='color: #e0e7ff; font-size: 12px;'>Strains</div>
        </div>
    </div>
</div>
""")

# Section headers
db_section_html = widgets.HTML("""
<div class='pheno-section'>
    <div class='pheno-header'>
        <span class='pheno-icon'>📚</span>
        Strain Database Management
    </div>
    <div class='pheno-description'>
        Import COA files, add custom strains, and build your personalized breeding library.
    </div>
</div>
""")

import_section_html = widgets.HTML("""
<div class='pheno-section'>
    <div class='pheno-header'>
        <span class='pheno-icon'>📤</span>
        Import Laboratory Data
    </div>
    <div class='pheno-description'>
        Upload CSV, JSON, or PDF files containing strain chemical profiles from lab testing.
    </div>
</div>
""")

manual_section_html = widgets.HTML("""
<div class='pheno-section'>
    <div class='pheno-header'>
        <span class='pheno-icon'>✍️</span>
        Manual Strain Entry
    </div>
    <div class='pheno-description'>
        Manually input strain data with complete cannabinoid and terpene profiles.
    </div>
</div>
""")

validation_section_html = widgets.HTML("""
<div class='pheno-section'>
    <div class='pheno-header'>
        <span class='pheno-icon'>🔬</span>
        Cross Validation & Accuracy Testing
    </div>
    <div class='pheno-description'>
        Test prediction accuracy by comparing AI predictions with known offspring data.
    </div>
</div>
""")

breeding_section_html = widgets.HTML("""
<div class='pheno-section'>
    <div class='pheno-header'>
        <span class='pheno-icon'>🧬</span>
        Breeding Optimization Engine
    </div>
    <div class='pheno-description'>
        AI-powered breeding recommendations based on target traits and genetic compatibility.
    </div>
</div>
""")

# Styled upload widget container
upload_container = widgets.VBox([
    upload_widget,
    widgets.HTML("<p style='color: #718096; font-size: 13px; margin-top: 10px;'>Supported formats: CSV, JSON, PDF</p>")
], layout=widgets.Layout(margin='10px 0'))

# Manual entry widgets with better styling
manual_strain_name = widgets.Text(
    description='Strain Name:',
    placeholder='e.g., Purple Kush',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='100%', margin='5px 0')
)

manual_type = widgets.Dropdown(
    options=['hybrid', 'indica', 'sativa', 'unknown'],
    value='hybrid',
    description='Strain Type:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='50%', margin='5px 0')
)

# Cannabinoid inputs in a cleaner grid
cannabinoid_header = widgets.HTML("""
<div style='font-weight: 600; color: #4a5568; margin: 20px 0 10px 0; font-size: 15px;'>
    💊 Cannabinoid Profile (%)
</div>
""")

manual_thc = widgets.FloatText(description='THC:', value=0.0, style={'description_width': '80px'}, layout=widgets.Layout(width='180px'))
manual_cbd = widgets.FloatText(description='CBD:', value=0.0, style={'description_width': '80px'}, layout=widgets.Layout(width='180px'))
manual_cbg = widgets.FloatText(description='CBG:', value=0.0, style={'description_width': '80px'}, layout=widgets.Layout(width='180px'))
manual_cbc = widgets.FloatText(description='CBC:', value=0.0, style={'description_width': '80px'}, layout=widgets.Layout(width='180px'))
manual_cbda = widgets.FloatText(description='CBDA:', value=0.0, style={'description_width': '80px'}, layout=widgets.Layout(width='180px'))

cannabinoid_grid = widgets.GridBox([manual_thc, manual_cbd, manual_cbg, manual_cbc, manual_cbda],
                                   layout=widgets.Layout(grid_template_columns='repeat(3, 1fr)', grid_gap='10px'))

# Terpene inputs in a cleaner grid
terpene_header = widgets.HTML("""
<div style='font-weight: 600; color: #4a5568; margin: 20px 0 10px 0; font-size: 15px;'>
    🌿 Terpene Profile (%)
</div>
""")

manual_myrcene = widgets.FloatText(description='Myrcene:', value=0.0, style={'description_width': '100px'}, layout=widgets.Layout(width='200px'))
manual_limonene = widgets.FloatText(description='Limonene:', value=0.0, style={'description_width': '100px'}, layout=widgets.Layout(width='200px'))
manual_pinene = widgets.FloatText(description='Pinene:', value=0.0, style={'description_width': '100px'}, layout=widgets.Layout(width='200px'))
manual_linalool = widgets.FloatText(description='Linalool:', value=0.0, style={'description_width': '100px'}, layout=widgets.Layout(width='200px'))
manual_caryophyllene = widgets.FloatText(description='Caryophyllene:', value=0.0, style={'description_width': '100px'}, layout=widgets.Layout(width='200px'))
manual_humulene = widgets.FloatText(description='Humulene:', value=0.0, style={'description_width': '100px'}, layout=widgets.Layout(width='200px'))

terpene_grid = widgets.GridBox([manual_myrcene, manual_limonene, manual_pinene, manual_linalool, manual_caryophyllene, manual_humulene],
                               layout=widgets.Layout(grid_template_columns='repeat(3, 1fr)', grid_gap='10px'))

# Breeding widgets with better styling
mode_header = widgets.HTML("""
<div style='font-weight: 600; color: #4a5568; margin: 20px 0 10px 0; font-size: 15px;'>
    🔄 Breeding Mode Selection
</div>
""")

quick_cross_toggle = widgets.ToggleButton(
    value=False,
    description='Quick Cross Mode',
    button_style='info',
    icon='random',
    layout=widgets.Layout(width='220px', height='40px'),
    style={'font_weight': '600'}
)

mode_description = widgets.HTML("""
<p style='color: #718096; font-size: 13px; margin: 10px 0 20px 0;'>
    <b>Trait Optimization:</b> AI selects best partners for your target trait<br>
    <b>Quick Cross:</b> Directly cross two selected parents
</p>
""")

parent_header = widgets.HTML("""
<div style='font-weight: 600; color: #4a5568; margin: 20px 0 10px 0; font-size: 15px;'>
    🧬 Parent Strain Selection
</div>
""")

main_parent_select = widgets.Dropdown(
    options=sorted(df['strain_name'].tolist()),
    description='Main Parent:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='100%', margin='5px 0')
)

second_parent_select = widgets.Dropdown(
    options=sorted(df['strain_name'].tolist()),
    description='Second Parent:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='100%', margin='5px 0'),
    disabled=True
)

trait_select = widgets.Dropdown(
    options=[
        ('🔥 Increase Potency', 'potency'),
        ('💊 Enhance Pain Relief', 'pain_relief'),
        ('😌 Reduce Anxiety', 'anxiety_relief'),
        ('🎯 Improve Focus', 'focus'),
        ('😴 Aid Sleep', 'sleep'),
        ('🩹 Reduce Inflammation', 'inflammation')
    ],
    description='Target Trait:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='100%', margin='5px 0')
)

trait_bias_slider = widgets.FloatSlider(
    value=0.7,
    min=0.5,
    max=0.9,
    step=0.05,
    description='Main Influence:',
    continuous_update=False,
    readout_format='.0%',
    layout=widgets.Layout(width='100%', margin='15px 0'),
    style={'description_width': '140px'}
)

# Action buttons with icons
add_manual_strain_button = widgets.Button(
    description='Add Strain',
    button_style='success',
    icon='plus',
    layout=widgets.Layout(width='200px', height='45px', margin='10px 5px'),
    style={'font_weight': '600'}
)

generate_button = widgets.Button(
    description="Generate Cross",
    button_style='success',
    icon='rocket',
    layout=widgets.Layout(width='240px', height='50px', margin='10px 5px'),
    style={'font_weight': '700'}
)

export_button = widgets.Button(
    description="Export Results",
    button_style='info',
    icon='download',
    layout=widgets.Layout(width='200px', height='45px', margin='10px 5px'),
    disabled=True,
    style={'font_weight': '600'}
)

validate_cross_button = widgets.Button(
    description='Validate Cross',
    button_style='warning',
    icon='check',
    layout=widgets.Layout(width='200px', height='45px', margin='10px 5px'),
    style={'font_weight': '600'}
)

export_db_button = widgets.Button(
    description='Export Database',
    button_style='primary',
    icon='database',
    layout=widgets.Layout(width='200px', height='45px', margin='10px 5px'),
    style={'font_weight': '600'}
)

view_db_button = widgets.Button(
    description='View Stats',
    button_style='',
    icon='bar-chart',
    layout=widgets.Layout(width='180px', height='45px', margin='10px 5px'),
    style={'font_weight': '600'}
)

# Validation widgets
validation_header = widgets.HTML("""
<div style='font-weight: 600; color: #4a5568; margin: 20px 0 10px 0; font-size: 15px;'>
    Select strains for accuracy validation
</div>
""")

validate_parent1 = widgets.Dropdown(
    options=[],
    description='Parent 1:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='32%', margin='5px')
)

validate_parent2 = widgets.Dropdown(
    options=[],
    description='Parent 2:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='32%', margin='5px')
)

validate_offspring = widgets.Dropdown(
    options=[],
    description='Known Offspring:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='35%', margin='5px')
)

# Output widgets with styling
import_output = widgets.Output(layout=widgets.Layout(
    border='1px solid #e2e8f0',
    border_radius='8px',
    padding='15px',
    margin='10px 0'
))

validation_output = widgets.Output(layout=widgets.Layout(
    border='1px solid #e2e8f0',
    border_radius='8px',
    padding='15px',
    margin='10px 0'
))

results_output = widgets.Output(layout=widgets.Layout(
    border='1px solid #e2e8f0',
    border_radius='8px',
    padding='15px',
    margin='10px 0',
    min_height='200px'
))

visualization_output = widgets.Output(layout=widgets.Layout(
    margin='10px 0'
))

# Tab interface for better organization
tab_contents = []

# Database Tab
db_tab = widgets.VBox([
    import_section_html,
    upload_container,
    manual_section_html,
    manual_strain_name,
    manual_type,
    cannabinoid_header,
    cannabinoid_grid,
    terpene_header,
    terpene_grid,
    add_manual_strain_button,
    validation_section_html,
    validation_header,
    widgets.HBox([validate_parent1, validate_parent2, validate_offspring]),
    validate_cross_button,
    widgets.HTML("<div style='margin: 20px 0;'></div>"),
    widgets.HBox([export_db_button, view_db_button]),
    import_output,
    validation_output
])

# Breeding Tab
breeding_tab = widgets.VBox([
    breeding_section_html,
    mode_header,
    quick_cross_toggle,
    mode_description,
    parent_header,
    main_parent_select,
    second_parent_select,
    trait_bias_slider,
    trait_select,
    widgets.HTML("<div style='margin: 20px 0;'></div>"),
    widgets.HBox([generate_button, export_button]),
    results_output,
    visualization_output
])

tabs = widgets.Tab([db_tab, breeding_tab])
tabs.set_title(0, '📚 Database')
tabs.set_title(1, '🧬 Breeding')

# ======================================================================
# DISPLAY UI
# ======================================================================

display(title_html)
display(tabs)

# ======================================================================
# EVENT HANDLERS
# ======================================================================

def update_strain_dropdowns():
    strain_list = sorted(strain_db.df['strain_name'].tolist())
    main_parent_select.options = strain_list
    second_parent_select.options = strain_list
    validate_parent1.options = strain_list
    validate_parent2.options = strain_list
    validate_offspring.options = strain_list

def on_quick_cross_toggle(change):
    if change['new']:
        trait_select.disabled = True
        second_parent_select.disabled = False
        trait_bias_slider.description = 'First Parent Influence:'
        trait_select.layout.visibility = 'hidden'
    else:
        trait_select.disabled = False
        second_parent_select.disabled = True
        trait_bias_slider.description = 'Main Parent Influence:'
        trait_select.layout.visibility = 'visible'

quick_cross_toggle.observe(on_quick_cross_toggle, names='value')

def on_upload(change):
    with import_output:
        clear_output(wait=True)
        print("="*70)
        print("IMPORTING COA DATA")
        print("="*70)

        for filename, file_info in upload_widget.value.items():
            print(f"\nProcessing: {filename}")

            temp_path = f"/tmp/{filename}"
            with open(temp_path, 'wb') as f:
                f.write(file_info['content'])

            success = strain_db.import_from_file(temp_path)

            if success:
                global df, X, X_tensor, ae
                df = strain_db.df
                X = df[trained_feature_columns].fillna(0)
                X_tensor = torch.tensor(X.values, dtype=torch.float32)

                print("\n🔄 Retraining model...")
                ae = Autoencoder(input_dim=len(trained_feature_columns))
                optimizer_new = optim.Adam(ae.parameters(), lr=0.005, weight_decay=1e-5)

                for epoch in range(100):
                    optimizer_new.zero_grad()
                    output = ae(X_tensor)
                    loss = criterion(output, X_tensor)
                    loss.backward()
                    optimizer_new.step()

                print(f"✓ Model retrained (Loss: {loss.item():.4f})")
                update_strain_dropdowns()

        print(f"\n{'='*70}")
        print(f"DATABASE UPDATED")
        print(f"Total Strains: {len(strain_db.df)}")
        print(f"{'='*70}")

upload_widget.observe(on_upload, names='value')

def add_manual_strain(b):
    with import_output:
        clear_output(wait=True)

        if not manual_strain_name.value:
            print("❌ Please enter a strain name")
            return

        strain_data = {
            'strain_name': manual_strain_name.value,
            'type': manual_type.value,
            'thc_pct': manual_thc.value,
            'cbd_pct': manual_cbd.value,
            'cbg_pct': manual_cbg.value,
            'cbc_pct': manual_cbc.value,
            'cbda_pct': manual_cbda.value,
            'myrcene_pct': manual_myrcene.value,
            'limonene_pct': manual_limonene.value,
            'pinene_pct': manual_pinene.value,
            'linalool_pct': manual_linalool.value,
            'caryophyllene_pct': manual_caryophyllene.value,
            'humulene_pct': manual_humulene.value
        }

        strain_db.add_strain(strain_data)

        global df, X, X_tensor, ae
        df = strain_db.df
        X = df[trained_feature_columns].fillna(0)
        X_tensor = torch.tensor(X.values, dtype=torch.float32)

        print("\n🔄 Retraining model...")
        ae = Autoencoder(input_dim=len(trained_feature_columns))
        optimizer_new = optim.Adam(ae.parameters(), lr=0.005, weight_decay=1e-5)

        for epoch in range(100):
            optimizer_new.zero_grad()
            output = ae(X_tensor)
            loss = criterion(output, X_tensor)
            loss.backward()
            optimizer_new.step()

        print(f"✓ Model retrained (Loss: {loss.item():.4f})")
        update_strain_dropdowns()

        manual_strain_name.value = ''
        manual_thc.value = 0.0
        manual_cbd.value = 0.0

        print(f"\n✅ Strain added successfully!")

add_manual_strain_button.on_click(add_manual_strain)

def generate_optimized_cross(b):
    global generated_results

    with results_output:
        clear_output(wait=True)
        print("="*70)
        print("GENERATING HYBRID STRAIN")
        print("="*70)

        main_parent = main_parent_select.value
        trait_bias = trait_bias_slider.value

        if not main_parent:
            print("\n❌ Please select a main parent strain")
            return

        breeder = AdvancedBreedingStrategy(df, ae)
        visualizer = StrainVisualizer()

        if quick_cross_toggle.value:
            second_parent = second_parent_select.value
            if not second_parent:
                print("\n❌ Please select a second parent strain")
                return

            print(f"\n🧬 First Parent:  {main_parent}")
            print(f"🧬 Second Parent: {second_parent}")
            print(f"⚖️ First Parent Influence: {trait_bias:.2f}")

            result = breeder.optimize_cross(main_parent, second_parent, 'potency')
            results = [{
                'partner': second_parent,
                'profile': result['chemical_profile'],
                'predicted_traits': result['predicted_traits']
            }]

            with visualization_output:
                clear_output(wait=True)
                profile_df = pd.DataFrame(result['chemical_profile'], columns=trained_feature_columns)
                fig_chem = visualizer.plot_chemical_profile(profile_df.iloc[0], f"Offspring: {main_parent} × {second_parent}")
                fig_chem.show()

            print("\n🎯 PREDICTED TRAITS")
            print("-"*70)
            for trait, score in result['predicted_traits'].items():
                bar = "█" * int(score * 2) + "░" * (20 - int(score * 2))
                print(f"  {trait:<15}: {score:6.2f}  [{bar}]")

        else:
            desired_trait = trait_select.value
            optimizer = TraitOptimizer(df)

            print(f"\n🧬 Main Parent: {main_parent}")
            print(f"🎯 Target Trait: {desired_trait}")

            best_partners = optimizer.find_optimal_partners(main_parent, desired_trait)

            print("\n📊 Top Partners:")
            print("-"*70)
            for partner, score in best_partners:
                print(f"  • {partner:<30} (Score: {score:.3f})")

            results = []
            for partner, _ in best_partners[:3]:
                result = breeder.optimize_cross(main_parent, partner, desired_trait)
                results.append({
                    'partner': partner,
                    'profile': result['chemical_profile'],
                    'predicted_traits': result['predicted_traits']
                })

            with visualization_output:
                clear_output(wait=True)
                fig_traits = visualizer.plot_trait_comparison(main_parent, best_partners[:3], optimizer)
                fig_traits.show()

        print("\n🔬 CROSS ANALYSIS")
        print("="*70)
        for result_item in results:
            print(f"\n💠 {main_parent} × {result_item['partner']}")
            print("-"*70)
            for trait, score in result_item['predicted_traits'].items():
                bar = "█" * int(score * 2) + "░" * (20 - int(score * 2))
                print(f"  {trait:<15}: {score:6.2f}  [{bar}]")

        generated_results = results
        export_button.disabled = False
        print("\n✅ Generation complete!")

generate_button.on_click(generate_optimized_cross)

def export_results(b):
    with results_output:
        if not generated_results:
            print("\n❌ No results to export")
            return

        export_data = []
        for result in generated_results:
            row = {
                'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'main_parent': main_parent_select.value,
                'partner': result['partner']
            }

            profile_df = pd.DataFrame(result['profile'], columns=trained_feature_columns)
            for col in trained_feature_columns:
                row[col] = profile_df.iloc[0][col]

            for trait, score in result['predicted_traits'].items():
                row[f'trait_{trait}'] = score

            export_data.append(row)

        export_df = pd.DataFrame(export_data)
        filename = f"pheno_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        export_df.to_csv(filename, index=False)

        print(f"\n✅ Exported to {filename}")

export_button.on_click(export_results)

def validate_cross(b):
    with validation_output:
        clear_output(wait=True)

        if not all([validate_parent1.value, validate_parent2.value, validate_offspring.value]):
            print("❌ Please select all strains")
            return

        result = strain_db.validate_cross_accuracy(validate_parent1.value, validate_parent2.value, validate_offspring.value)

validate_cross_button.on_click(validate_cross)

def export_database(b):
    with import_output:
        clear_output(wait=True)
        filename = strain_db.export_database()
        print(f"\n✅ Database exported to {filename}")

export_db_button.on_click(export_database)

def view_database_stats(b):
    with import_output:
        clear_output(wait=True)
        print("="*70)
        print("DATABASE STATISTICS")
        print("="*70)
        print(f"\n📊 Total Strains: {len(strain_db.df)}")
        print(f"🌱 Original: {strain_db.original_size}")
        print(f"➕ Custom: {len(strain_db.custom_strains)}")

view_db_button.on_click(view_database_stats)

# ======================================================================
# DISPLAY UI
# ======================================================================

display(title_html)

display(widgets.HTML("<h2 style='color: #43e97b;'>📚 Database Management</h2>"))
display(widgets.HTML("<h3>📤 Import COA Data</h3>"))
display(upload_widget)

display(widgets.HTML("<h3>✍️ Manual Strain Entry</h3>"))
display(manual_strain_name)
display(manual_type)

display(widgets.HTML("<h4>Cannabinoids</h4>"))
display(widgets.HBox([widgets.VBox([manual_thc, manual_cbg, manual_cbda]), widgets.VBox([manual_cbd, manual_cbc])]))

display(widgets.HTML("<h4>Terpenes</h4>"))
display(widgets.HBox([widgets.VBox([manual_myrcene, manual_limonene, manual_pinene]), widgets.VBox([manual_linalool, manual_caryophyllene, manual_humulene])]))

display(add_manual_strain_button)

display(widgets.HTML("<h3>🔬 Cross Validation</h3>"))
display(widgets.HBox([validate_parent1, validate_parent2, validate_offspring]))
display(validate_cross_button)

display(widgets.HTML("<h3>💾 Database Actions</h3>"))
display(widgets.HBox([export_db_button, view_db_button]))

display(widgets.HTML("<h3>📊 Results</h3>"))
display(import_output)
display(validation_output)

display(widgets.HTML("<h2 style='color: #667eea;'>🧬 Breeding Optimization</h2>"))
display(widgets.HTML("<h3>🔄 Mode</h3>"))
display(quick_cross_toggle)

display(widgets.HTML("<h3>🧬 Parent Selection</h3>"))
display(main_parent_select)
display(second_parent_select)
display(trait_bias_slider)
display(trait_select)

display(widgets.HBox([generate_button, export_button]))

display(widgets.HTML("<h3>📊 Analysis & Results</h3>"))
display(results_output)
display(visualization_output)

update_strain_dropdowns()

# Success message with better styling
print("\n" + "="*70)
print("🎉 Cannabis PhenoHunter Pro - Successfully Initialized!")
print("="*70)
print(f"\n✅ System Status: OPERATIONAL")
print(f"📊 Database: {len(strain_db.df)} strains loaded")
print(f"🧠 AI Model: Trained and ready")
print(f"🔬 Analysis Tools: Active\n")
print("="*70)
print("\n💡 Quick Start:")
print("  1. Navigate to the 'Database' tab to import strains")
print("  2. Switch to 'Breeding' tab to generate crosses")
print("  3. Use validation tools to test accuracy")
print("\n" + "="*70)


CANNABIS PHENO HUNTER PRO - ENHANCED EDITION
Advanced Trait-Based Breeding & Real Data Integration
✓ Loaded 10 premium strains with complete chemical profiles
✓ Average THC: 21.53%, CBD: 0.55%

Training autoencoder...
  Epoch 50/200 - Loss: 6.0918
  Epoch 100/200 - Loss: 3.7724
  Epoch 150/200 - Loss: 5.2752
  Epoch 200/200 - Loss: 2.6760
✓ Autoencoder trained successfully
✓ Strain database manager initialized


HTML(value="\n\n<style>\n    .pheno-section {\n        background: white;\n        padding: 20px;\n        bor…

HTML(value="\n\n<style>\n    .pheno-section {\n        background: white;\n        padding: 20px;\n        bor…

HTML(value="<h2 style='color: #43e97b;'>📚 Database Management</h2>")

HTML(value='<h3>📤 Import COA Data</h3>')

...

Dropdown(description='Target Trait:', layout=Layout(margin='5px 0', width='100%'), options=(('🔥 Increase Poten…

HTML(value='<h3>📊 Analysis & Results</h3>')

Output(layout=Layout(border='1px solid #e2e8f0', margin='10px 0', min_height='200px', padding='15px'))

Output(layout=Layout(margin='10px 0'))


🎉 Cannabis PhenoHunter Pro - Successfully Initialized!

✅ System Status: OPERATIONAL
📊 Database: 10 strains loaded
🧠 AI Model: Trained and ready
🔬 Analysis Tools: Active


💡 Quick Start:
  1. Navigate to the 'Database' tab to import strains
  2. Switch to 'Breeding' tab to generate crosses
  3. Use validation tools to test accuracy

